## Battery Aging Challenge
Estimating the remaining capacity form field data.

### Input
* 6 months of battery voltage, battery current and battery temperature (stored in monthly .csv files: 2020-04 to 2020-10)
* regular testbench capacity measurements for the first 3 months
* chemistry: NMC

### Task
Estimate the remaining capacity at the following dates:
* 2020-07-11
* 2020-08-04
* 2020-08-28
* 2020-09-22
* 2020-10-01

### Gotcha
Capacity test related time series were removed from the data to prevent the solution of plain coulomb counting for those tests. Therefore you will recognize data gaps at certain dates.

In [ ]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from utils import plot_battery_data, plot_testbench_results
import numpy as np
%matplotlib widget  
from ipywidgets import *
import matplotlib.pyplot as plt


In [ ]:
# load dataset
battery_data = pd.concat([pd.read_csv('data/battery_data/2020-{}.csv'.format(month)) for month in range(4, 10)])
battery_data['datetime'] = pd.to_datetime(battery_data['timestamp'], unit='s')
battery_data['power'] = battery_data['current'] * battery_data['voltage']
# extract and plot one month.
subset = battery_data[battery_data['timestamp']<datetime.datetime(2020, 4, 29, 0, 0, 0).timestamp()]
plt.figure()
key = 'voltage'
plt.plot(subset['datetime'], subset[key])
plt.show()
# Use this graph to identify the motif you want to search for. Enter the start and end date and time for your motif in the next cell

In [ ]:
# Enter the start and end date and time for your motif here. Note that for some reason you need to add one hour on
# here are some premade motifs

# voltage hour
# motif = battery_data[(datetime.datetime(2020, 4, 2, 12, 30, 47).timestamp() < battery_data['timestamp']) &
#                      (datetime.datetime(2020, 4, 2, 13, 33, 57).timestamp() > battery_data['timestamp'])]

# voltage charge
# so here we are just searching for the charge part of the cycle
motif = battery_data[(datetime.datetime(2020, 4, 2, 15, 26, 50).timestamp() < battery_data['timestamp']) &
                     (datetime.datetime(2020, 4, 2, 15, 34, 38).timestamp() > battery_data['timestamp'])]

# power charging
# motif = battery_data[(datetime.datetime(2020, 4, 2, 13, 29, 38).timestamp() < battery_data['timestamp']) &
#                      (datetime.datetime(2020, 4, 2, 13, 33, 56).timestamp() > battery_data['timestamp'])]

# current hour
# motif = battery_data[(datetime.datetime(2020, 4, 2, 12, 30, 53).timestamp() < battery_data['timestamp']) &
#                      (datetime.datetime(2020, 4, 2, 13, 34, 8).timestamp() > battery_data['timestamp'])]

# day
# motif = battery_data[(datetime.datetime(2020, 4, 2, 11, 34, 3).timestamp() < battery_data['timestamp']) &
#                      (datetime.datetime(2020, 4, 3, 5, 1, 17).timestamp() > battery_data['timestamp'])]


# The data is very high resolution, so it can be slow to search. Best to reduce resolution with sampling
# normalise the motif to mean of zero for comparison
sampling = 5
# plot the motif
plt.figure()
data, motif = battery_data[::sampling][key].to_numpy(), motif[::sampling][key].to_numpy()
plt.plot(motif)

In [ ]:

motif_norm = motif - np.mean(motif)

l = len(motif)
errors = []
for start in range(data.shape[0] - l):
    samp = data[start:start + l] - np.mean(data[start:start + l])
    errors.append(np.mean((samp - motif_norm)**2))

In [ ]:
plt.figure()
plt.plot(errors)
plt.show()

In [ ]:
error_cutoff = 0.0001
dataset = []
prev_start = - l
n = 0
fig, axs = plt.subplots(3)
fig.set_size_inches(12, 10)
for start, error in enumerate(errors):
    if (error < error_cutoff) & (start - l >= prev_start):
        prev_start = start
        c = start/len(errors)
        # plt.plot(data[start:start + l] - np.mean(data[start:start + l]) - motif, color = (c, 1-c, 0), alpha=0.02)
        for ax, key in zip(axs.ravel(), ['current', 'voltage', 'power']):
            data = battery_data[key][::sampling].to_numpy()
            ax.plot(data[start:start + l], color = (c, 1-c, 1-c), alpha=0.01)
            ax.set_ylabel(key)

        # plt.plot(, color = (1, 1, 0), alpha=1)
        dataset.append(data[start:start + l])
        n+=1
print(n)
plt.show()


In [ ]:

plt.figure()
plt.imshow(np.array(dataset - np.mean(dataset, axis=0)))

In [ ]:
plt.figure()
step = 100
for s in range(0, n-step, step):
    plt.plot(abs(np.mean(dataset[s:s+step] - np.mean(dataset, axis=0), axis=0)), c =( s/887, 1-s/887, 1-s/887), linewidth=0.7)
    # plt.plot(np.mean(dataset[s:s+step] - np.mean(dataset, axis=0), axis=0), c =( s/887, 1-s/887, 1-s/887), linewidth=0.7)
